# Object Detection with YOLOv8 - Demo Notebook

This notebook demonstrates how to use the object detection system for:
- Loading and using pre-trained models
- Training custom models
- Running inference on images and videos
- Evaluating model performance

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install ultralytics opencv-python matplotlib seaborn pandas

In [ ]:
# Import libraries
import sys
sys.path.append('../src')

from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load Pre-trained Model

In [ ]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # nano model (fastest)
# model = YOLO('yolov8s.pt')  # small model
# model = YOLO('yolov8m.pt')  # medium model
# model = YOLO('yolov8l.pt')  # large model
# model = YOLO('yolov8x.pt')  # xlarge model (most accurate)

print("Model loaded successfully!")
print(f"Classes: {len(model.names)}")

## 3. Image Detection

In [ ]:
# Download sample image
!wget -O sample.jpg https://ultralytics.com/images/bus.jpg

# Run detection
results = model.predict(
    source='sample.jpg',
    conf=0.25,
    iou=0.45,
    save=False
)

# Display results
annotated_image = results[0].plot()
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Object Detection Results')
plt.show()

# Print detections
print(f"\nDetected {len(results[0].boxes)} objects:")
for box in results[0].boxes:
    cls = int(box.cls[0])
    conf = float(box.conf[0])
    print(f"  - {model.names[cls]}: {conf:.2f}")

## 4. Video Detection

In [ ]:
# Process video
video_path = '../data/videos/sample.mp4'

# Run detection on video
results = model.predict(
    source=video_path,
    conf=0.25,
    save=True,
    project='../outputs',
    name='video_detection'
)

print("Video processing complete!")
print(f"Output saved to: ../outputs/video_detection")

## 5. Webcam Detection (Real-time)

In [ ]:
# Note: This requires a webcam and may not work in all notebook environments
# For real-time detection, use the detect.py script instead

# Example command:
# !python ../src/detect.py --source 0 --weights yolov8n.pt

## 6. Training Custom Model

In [ ]:
# Prepare dataset in YOLO format
# Directory structure:
# dataset/
#   ├── images/
#   │   ├── train/
#   │   └── val/
#   └── labels/
#       ├── train/
#       └── val/

# Create dataset.yaml
dataset_yaml = """
path: ../data/datasets/custom
train: images/train
val: images/val

nc: 3
names:
  0: class1
  1: class2
  2: class3
"""

with open('../configs/custom_dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("Dataset configuration created!")

In [ ]:
# Train model
model = YOLO('yolov8n.pt')

results = model.train(
    data='../configs/custom_dataset.yaml',
    epochs=50,
    batch=16,
    imgsz=640,
    patience=10,
    save=True,
    project='../runs/train',
    name='custom_model'
)

print("Training complete!")

## 7. Model Evaluation

In [ ]:
# Load trained model
model = YOLO('../runs/train/custom_model/weights/best.pt')

# Evaluate on validation set
metrics = model.val(
    data='../configs/custom_dataset.yaml',
    conf=0.001,
    iou=0.6
)

# Print metrics
print(f"\nEvaluation Results:")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## 8. Visualize Training Results

In [ ]:
import pandas as pd

# Load training results
results_csv = '../runs/train/custom_model/results.csv'
df = pd.read_csv(results_csv)
df.columns = df.columns.str.strip()

# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Box Loss')
axes[0, 0].plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Losses')
axes[0, 0].legend()
axes[0, 0].grid(True)

# mAP curves
axes[0, 1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5')
axes[0, 1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('mAP')
axes[0, 1].set_title('Mean Average Precision')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Precision/Recall
axes[1, 0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
axes[1, 0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Precision and Recall')
axes[1, 0].legend()
axes[1, 0].grid(True)

# F1 Score
f1_score = 2 * (df['metrics/precision(B)'] * df['metrics/recall(B)']) / \
           (df['metrics/precision(B)'] + df['metrics/recall(B)'])
axes[1, 1].plot(df['epoch'], f1_score, label='F1 Score', color='green')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('F1 Score')
axes[1, 1].set_title('F1 Score')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 9. Export Model

In [ ]:
# Export to different formats
model = YOLO('../runs/train/custom_model/weights/best.pt')

# Export to ONNX
model.export(format='onnx', imgsz=640)

# Export to TorchScript
# model.export(format='torchscript', imgsz=640)

# Export to CoreML (for iOS)
# model.export(format='coreml', imgsz=640)

print("Model exported successfully!")

## 10. Batch Processing

In [ ]:
# Process multiple images
image_dir = '../data/images/'
output_dir = '../outputs/batch/'

results = model.predict(
    source=image_dir,
    conf=0.25,
    save=True,
    project=output_dir,
    name='batch_results'
)

print(f"Processed {len(results)} images")
print(f"Results saved to: {output_dir}")

## 11. Performance Benchmarking

In [ ]:
import time

# Benchmark inference speed
model = YOLO('yolov8n.pt')
dummy_image = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

# Warmup
for _ in range(10):
    _ = model.predict(dummy_image, verbose=False)

# Benchmark
iterations = 100
start_time = time.time()

for _ in range(iterations):
    _ = model.predict(dummy_image, verbose=False)

end_time = time.time()

avg_time = (end_time - start_time) / iterations
fps = 1 / avg_time

print(f"\nBenchmark Results:")
print(f"Average inference time: {avg_time*1000:.2f}ms")
print(f"FPS: {fps:.2f}")

## 12. Conclusion

This notebook demonstrated:
- Loading pre-trained YOLOv8 models
- Running inference on images and videos
- Training custom models
- Evaluating model performance
- Exporting models for deployment
- Performance benchmarking

For production deployment, use the provided API server (`src/api.py`) or command-line scripts.